In [19]:
from okx import PublicData

public = PublicData.PublicAPI(flag='0')
data = public.get_instruments('SWAP')['data']
names = [x['instId'] for x in data if  "USDT-SWAP" in x['instId']]

In [20]:
# from okx import TradingData

# api = TradingData.TradingDataAPI(flag='0')
# api.get_margin_lending_ratio(ccy="ETH")

In [21]:
import okx.MarketData as MarketData
import pandas as pd

market = MarketData.MarketAPI(flag='0')
def get_candles(instId='ETH-USDT-SWAP', bar='1H', limit="100"):
    result = market.get_candlesticks(instId=instId, bar=bar, limit=limit)

    data = result['data']  # 列表，每项：[ts, o, h, l, c, vol, volCcy, ...]
    df = pd.DataFrame(data, columns=[
        'ts', 'open', 'high', 'low', 'close', 
        'vol', 'volCcy', 'volCcyQuote', 'confirm'
    ])
    df = df.loc[:, ["open", "high", "low", "close"]]
    df = df.astype({'open':float, 'high':float, 'low':float, 'close':float})
    df = df[::-1].reset_index(drop=True)    # 最后一行为新k

    return df

In [22]:
import pandas as pd
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

def tr(data):
    data['previous_close'] = data['close'].shift(1)
    data['high-low'] = abs(data['high'] - data['low'])
    data['high-pc'] = abs(data['high'] - data['previous_close'])
    data['low-pc'] = abs(data['low'] - data['previous_close'])

    tr = data[['high-low', 'high-pc', 'low-pc']].max(axis=1)

    return tr

def atr(data, period):
    data['tr'] = tr(data)
    atr = data['tr'].rolling(period).mean()

    return atr

def supertrend(df, period=10, atr_multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = atr(df, period)
    df['upperband'] = hl2 + (atr_multiplier * df['atr'])
    df['lowerband'] = hl2 - (atr_multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1

        if df['close'][current] > df['upperband'][previous]:
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            df['in_uptrend'][current] = False
        else:
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]

            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df

In [23]:
import time

def main():
    for name in names:
        time.sleep(0.5)
        candles_1H = get_candles(name, bar='1H', limit="200")
        supertrend_1H = supertrend(candles_1H)
        supertrend_1H = supertrend_1H["in_uptrend"]
        current_trend_1H = supertrend_1H.iloc[-1]
        prev_trend_1H = supertrend_1H.iloc[-2]


        # 上升趋势转为下跌趋势
        if current_trend_1H == False and prev_trend_1H == True:
            print(name, flush=True)


In [24]:
while True:
    main()
    print("main完成")

ATH-USDT-SWAP
HUMA-USDT-SWAP
PIEVERSE-USDT-SWAP
SOPH-USDT-SWAP
main完成
ATH-USDT-SWAP
HUMA-USDT-SWAP
ME-USDT-SWAP
PIEVERSE-USDT-SWAP


KeyboardInterrupt: 